In [1]:
import pandas as pd
import MySQLdb as sql

import plotly.offline as plotly
import plotly.graph_objs as go
plotly.init_notebook_mode(connected=True)

In [2]:
%matplotlib inline

# Get Data

## From Pickled Data

In [ ]:
# ?

## From SQL

In [3]:
sql_params = {
    'host': 'localhost',
    'port': 3306,
    'user': '',
    'passwd': '',
    'db': ''
}

In [4]:
# ===== Submission Table ===== #

db = sql.connect(**sql_params)
cur = db.cursor()
cur.execute('SELECT * FROM submissions;')
raw_data = cur.fetchall()
cur.close()
db.close()

columns = [
    'submission_id', 'submission_datetime', 'retrieval_datetime', 'score', 'num_comments', 'author_name', 'author_id',
    'upvote_ratio', 'url', 'permalink', 'subreddit_name', 'subreddit_id', 'title', 'text'
]

submissions = {}
for i in range(len(columns)):
    submissions[columns[i]] = [row[i] for row in raw_data]
    
submissions = pd.DataFrame(submissions)

# Using date instead of datetime here makes ploting easier
submissions['submission_date'] = submissions['submission_datetime'].map(lambda x: x.date())

In [5]:
# ===== Comment Table ===== #

db = sql.connect(**sql_params)
cur = db.cursor()
cur.execute('SELECT * FROM comments;')
raw_data = cur.fetchall()
cur.close()
db.close()

columns = [
    'comment_id', 'comment_datetime', 'retrieval_datetime', 'score', 'subreddit_name', 'subreddit_id', 'text',
    'author_name', 'author_id', 'parent_id', 'link_id'
]
comments = {}
for i in range(len(columns)):
    comments[columns[i]] = [row[i] for row in raw_data]
    
comments = pd.DataFrame(comments)

In [6]:
# Run this to create .pickle file
submissions.to_pickle('submissions.pickle')
comments.to_pickle('comments.pickle')

# Exploratory Analysis

In [145]:
request_posts = submissions[submissions['title'].map(lambda x: '[REQ]' in x)]
response_posts = submissions[submissions['title'].map(lambda x: '[PAID]' in x or '[UNPAID]' in x)]
paid_posts = submissions[submissions['title'].map(lambda x: '[PAID]' in x)]
unpaid_posts = submissions[submissions['title'].map(lambda x: '[UNPAID]' in x)]

In [146]:
# Popularity of the r/borrow over time

num_requests_per_day = request_posts.groupby('submission_date').size()
num_responses_per_day = response_posts.groupby('submission_date').size()

plotly.iplot([
    go.Scatter(
        x=num_requests_per_day.index,
        y=num_requests_per_day.rolling(10).mean(),
        name='num_requests'
    ),
    go.Scatter(
        x=num_responses_per_day.index,
        y=num_responses_per_day.rolling(10).mean(),
        name='num_responses'
    )
])

In [147]:
# Paid vs unpaid responses

paid_per_day = paid_posts.groupby('submission_date').size()
unpaid_per_day = unpaid_posts.groupby('submission_date').size()

plotly.iplot([
    go.Scatter(
        x=paid_per_day.index,
        y=paid_per_day.rolling(10).mean(),
        name='paid_responses'
    ),
    go.Scatter(
        x=unpaid_per_day.index,
        y=unpaid_per_day.rolling(10).mean(),
        name='unpaid_responses'
    )
])

In [148]:
# Repayment rate

paid_per_day = paid_posts.groupby('submission_date').size()
unpaid_per_day = unpaid_posts.groupby('submission_date').size()
payment_proportion = paid_per_day / (unpaid_per_day + paid_per_day)

plotly.iplot([
    go.Scatter(
        x=payment_proportion.index,
        y=payment_proportion.interpolate().rolling(20).mean(),
        name='paid_responses'
    )
])

In [159]:
# Proportion of requests that follow the stricter naming standards
# Inferred from the presence of a '(#' string for the location

request_posts.loc[:,'valid_format'] = request_posts.loc[:,'title'].map(lambda x: '(#' in x)
valid_prop = (
    request_posts.groupby('submission_date')['valid_format'].sum() /
    request_posts.groupby('submission_date').size()
)

plotly.iplot([
    go.Scatter(
        x=valid_prop.index,
        y=valid_prop.rolling(5).mean()
    )
])

# Parser

Some experimental code and spot checks used to write the request parser found in [here](https://github.com/borrowbot/service_submission_parser). A notebook with more indepth parse-quality analysis is being worked on for check in and detailed documentation on the parsing logic and known failure cases is included with parser repository.

In [121]:
class ParseNoTag(Exception):
    pass


class RequestInvalidLocation(Exception):
    pass


class RequestInvalidFinancials(Exception):
    pass

In [96]:
class PostParser(object):
    @staticmethod
    def parse_submission(submission_row):
        title = submission_row['title'].lower()
        
        n_tags = sum([tag in title for tag in ['[req]', '[paid]', '[unpaid]']])
        if n_tags != 1:
            raise ParseNoTag
        
        if '[req]' in title:
            return ParseRequest.parse_request(title)

## Response Parser

In [42]:
class ParseResponse(object):
    @staticmethod
    def parse_response(submission):
        pass

## Request Parser

In [296]:
import re


class ParseRequest(object):
    prearrange_keywords = ['prearrange', 'pre arrange', 'pre-arrange']
    paypal_keywords = ['paypal', 'pay pal', 'pay-pal']
    venmo_keywords = ['venmo']
    cashapp_keywords = ['cashapp', 'cash app', 'cash-app']
    zelle_keywords = ['zelle']
    
    _prospective_payment_types = [
        'bank transfer', 'bank-transfer','bt',
        'square',
        'google wallet', 'google-wallet',
        'apple pay', 'applepay', 'apple-pay',
        'crypto', 'btc', 'bitcoin', 'bit-coin', 'bit coin',
        'western union', 'western-union',
        'e-transfer', 'interac', 'etransfer', 'e transfer',
        'monzo'
    ]
    
    repayment_kws = [
        'jan', 'january',
        'feb', 'february',
        'mar', 'march',
        'apr', 'april',
        'may',
        'jun', 'june',
        'jul', 'july',
        'aug', 'august',
        'sept', 'september',
        'oct', 'october',
        'nov', 'november',
        'dec', 'december',
        'week',
        'month',
        'mon', 'monday',
        'tues', 'tuesday',
        'wed', 'wednesday',
        'thur', 'thursday',
        'fri', 'friday',
        'sat', 'saturday',
        'sun', 'sunday'
    ]
    
    
    @staticmethod
    def parse_request(submission):
        location = ParseRequest.get_location(submission)
        prearranged = ParseRequest.find_keyword(submission, ParseRequest.prearrange_keywords)
        payment_types = ParseRequest.get_payment_types(submission)
        repay_date = ParseRequest.find_repay_date(submission)
        principal, repay, currency = ParseRequest.get_loan_amounts(submission)
        
        return {
            'raw_title': submission,
            'location': location,
            'prearranged': prearranged,
            'payment_types': payment_types,
            'repay_date': repay_date,
            'currency': currency,
            'principal': principal,
            'repay': repay
        }
    
    
    @staticmethod
    def find_keyword(submission, kw):
        return any([k in submission for k in kw])
        
    
    @staticmethod
    def get_marked_numbers(marker, s, buffer_chars=' '):
        blocks = s.replace(',', '').split(marker)

        after_blocks = [re.match('[{}]*[0-9]+'.format(buffer_chars), block) for block in blocks[1:]]
        after_blocks = [
            int(re.sub('[{}]'.format(buffer_chars), '', block.group()))
            for block in after_blocks if block is not None
        ]

        before_blocks = [re.match('[{}]*[0-9]+'.format(buffer_chars), block[::-1]) for block in blocks[:-1]]
        before_blocks = [
            int(re.sub('[{}]'.format(buffer_chars), '', block.group()[::-1]))
            for block in before_blocks if block is not None
        ]

        return before_blocks + after_blocks
    
    
    @staticmethod
    def check_good_money_amount(value):
        return value <= 1250 and value >= 40
        
    
    @staticmethod
    def check_good_location(location):
        if re.search('[0-9]', location) is not None:
            raise RequestInvalidLocation
        if len(location) < 2:
            raise RequestInvalidLocation

            
    @staticmethod
    def get_location(submission_title):
        parts = submission_title.split('(#')
        if len(parts) != 2:
            raise RequestInvalidLocation
            
        tentative_location = parts[1].split(')')[0]
        ParseRequest.check_good_location(tentative_location)
        
        return tentative_location
    
    
    @staticmethod
    def get_payment_types(submission):
        payment_types = []
        
        if ParseRequest.find_keyword(submission, ParseRequest.paypal_keywords):
            payment_types.append('paypal')
            
        if ParseRequest.find_keyword(submission, ParseRequest.venmo_keywords):
            payment_types.append('venmo')
                
        if ParseRequest.find_keyword(submission, ParseRequest.cashapp_keywords):
            payment_types.append('cashapp')
            
        if ParseRequest.find_keyword(submission, ParseRequest.zelle_keywords):
            payment_types.append('zelle')
            
        return payment_types
    

    @staticmethod
    def find_repay_date(s):
        candidates_slash = re.findall('[0-9]{1,}/[0-9]{1,}', s)
        candidates_dash = [c.replace('-', '/') for c in re.findall('[0-9]{1,}-[0-9]{1,}', s)]
        candidates = candidates_slash + candidates_dash

        candidates = [c for c in candidates if all([int(d) < 32 and int(d) > 0 for d in c.split('/')])]

        if len(candidates) != 1 or int(candidates[0].split('/')[0]) > 12:
            return None

        return candidates[0]
    
    
    @staticmethod
    def get_loan_amounts(submission):
        unmarked_cands = [int(match) for match in re.findall('[0-9]+', submission)]
        usd_cands = ParseRequest.get_marked_numbers('$', submission)
        can_cands = ParseRequest.get_marked_numbers('cad', submission, ' $')
        eur_cands = (
            ParseRequest.get_marked_numbers('eur', submission) +
            ParseRequest.get_marked_numbers('€', submission)
        )
        gbp_cands = (
            ParseRequest.get_marked_numbers('gbp', submission) +
            ParseRequest.get_marked_numbers('£', submission)
        )
        
        # raisees error if there is more than one currency
        if len(eur_cands) > 0 and len(gbp_cands) > 0:
            raise RequestCantParseFinancials
        if len(eur_cands) > 0 and (len(usd_cands) > 0 or len(can_cands) > 0):
            raise RequestCantParseFinancials
        if len(gbp_cands) > 0 and (len(usd_cands) > 0 or len(can_cands) > 0):
            raise RequestCantParseFinancials
        
        currency = 'usd'
        money = usd_cands
        if len(eur_cands) > 0:
            currency = 'eur'
            money = eur_cands
        if len(gbp_cands) > 0:
            currency = 'gbp'
            money = gbp_cands
        if len(can_cands) > 0:
            currency = 'cad'
            money = can_cands
            
        money = [b for b in list(set(money + unmarked_cands)) if ParseRequest.check_good_money_amount(b)]
        
        principal = None
        repay = None
        
        if len(money) == 1:
            principal = money[0]
            
        if len(money) == 2:
            if max(money) < min(money) * 1.25:
                principal = min(money)
                repay = max(money)
            elif min(money) < max(money) * 0.625:
                principal = max(money)
                
        if principal is not None and (principal < 50 or principal > 1000):
            principal = None
            repay = None
        
        return principal, repay, currency

## Parsing Testing

In [230]:
from datetime import datetime

In [297]:
failure_counter = {
    ParseNoTag: [],
    RequestInvalidLocation: [],
    RequestCantParseFinancials: []
}
parsed_requests = []


for entry in submissions[submissions['submission_datetime'] >= datetime.strptime('01/06/2016', '%d/%m/%Y')].tail(15000).index:
    try:
        req = PostParser.parse_submission(submissions.loc[entry])
        if req is not None:
            parsed_requests.append(req)
    except Exception as e:
        failure_counter[type(e)].append(submissions.loc[entry])

In [298]:
for r in parsed_requests:
    print(r['raw_title'])
    print(r['location'])
    print(r['prearranged'])
    print(r['payment_types'])
    print(r['currency'])
    print(r['principal'])
    print(r['repay'])
    print(r['repay_date'])
    print()

[req] ($75) - (#st. louis, mo, usa), (4/13), (paypal/pre-arranged)
st. louis, mo, usa
True
['paypal']
usd
75
None
4/13

[req] ($200) - (#cape town, western cape, south africa), ($225 on april first), (paypal)
cape town, western cape, south africa
False
['paypal']
usd
200
225
None

[req] ($250 ) - (#manhattan, kansas, usa), (04/06/2018), (paypal)
manhattan, kansas, usa
False
['paypal']
usd
250
None
04/06

[req] (250) - (#wallington,nj, usa), (3/28/17), (paypal)
wallington,nj, usa
False
['paypal']
usd
250
None
3/28

[req] ($200) - (#fargo, nd, usa) , ($225 on 5/25) , (paypal)
fargo, nd, usa
False
['paypal']
usd
200
225
5/25

[req] ($40) (#olyphant, pa, usa) (april-may) (paypal)
olyphant, pa, usa
False
['paypal']
usd
None
None
None

[req] ($900) - (#brooklyn, ny), (4/22), (paypal)
brooklyn, ny
False
['paypal']
usd
900
None
4/22

[req] (400$ cad) - (#montreal, qc, canada), (430$ cad on april 5th), (paypal, interac)
montreal, qc, canada
False
['paypal']
cad
400
430
None

[req] (amount ) - (

[req] (175) - (#henderson, nc, usa), (may 3rd repay date), (paypal/225 repay)
henderson, nc, usa
False
['paypal']
usd
None
None
None

[req] (120$ ) - (#buff, ny, us), (3/5), (pre-arranged paypal)
buff, ny, us
True
['paypal']
usd
120
None
3/5

[req] (£175) - (#guernsey, uk), (13/4/18), (pre-arranged)
guernsey, uk
True
[]
gbp
175
None
None

[req] ($250) - (#dallas, tx, usa), (4/16/2016), (paypal)
dallas, tx, usa
False
['paypal']
usd
250
None
4/16

[req] ($200) - (#nyc, new york, usa), ($215 on 4/6/2018), (pre-arranged)
nyc, new york, usa
True
[]
usd
200
215
4/6

[req] (80$) - (#boston, ny, usa), (15,4,2018), (paypal)
boston, ny, usa
False
['paypal']
usd
80
None
None

[req] (5000cad) - (#calgary, ab, canada), (mar 2019 500/mo+500 extra), (paypal)
calgary, ab, canada
False
['paypal']
cad
500
None
None

[req] ($30) - (#anchorage, ak, usa), ($40 on 4/13) (paypal
anchorage, ak, usa
False
['paypal']
usd
None
None
4/13

[req] ($175) - (#dallas, tx, usa), (4/13/18), ($185 paypal)
dallas, tx, usa

None

[req] ($20) - (#owensboro, ky,usa), (4/5/2018), (paypal)
owensboro, ky,usa
False
['paypal']
usd
None
None
4/5

[req] (£300) - (#portsmouth, hampshire, england), (28/04/2018 + 28/05/2018 + 28/06/2018, monthly instalments and negotiable interest), (paypal)
portsmouth, hampshire, england
False
['paypal']
gbp
300
None
None

[req] ($60) (#union mills,in,usa) (paypal)
union mills,in,usa
False
['paypal']
usd
60
None
None

[req] ($3,000) - (#seattle, washington, usa), (3-20-18), (paypal/venmo)
seattle, washington, usa
False
['paypal', 'venmo']
usd
None
None
3/20

[req] ($800) - (#toms river, nj, usa), (repayment on june 3rd), ($900 paypal - pre-arranged)
toms river, nj, usa
True
['paypal']
usd
800
900
None

[req] ($85) - (#grand island, hall, ne), (april 5th, 2018 ), (paypal)
grand island, hall, ne
False
['paypal']
usd
85
None
None

[req] ($150) - (#cherry hill, nj, usa), (05/15/2018), (paypal preferably)
cherry hill, nj, usa
False
['paypal']
usd
150
None
05/15

[req] ($700aud) - (#sydne

False
['paypal']
gbp
125
None
None

[req] ($150) - (#alameda, ca, usa), (4/13/2018), (paypal, venmo)
alameda, ca, usa
False
['paypal', 'venmo']
usd
150
None
4/13

[req] ($115) - (#atlanta, ga, usa), (4/13, 20% interest), (paypal)
atlanta, ga, usa
False
['paypal']
usd
115
None
4/13

[req] (150 ) - (#rotterdam the netherlans), (26-418), (pre-arranged)
rotterdam the netherlans
True
[]
usd
418
None
None

[req] ($60 repay $65) - (#brewton, alabama, usa), (5/3/2018), (paypal)
brewton, alabama, usa
False
['paypal']
usd
60
65
5/3

[req] (4500) - (#paris, île-de-france, france), (1400/mo 4mo 4/28 - 7/28), (paypal, cash, any..) (collateral available)
paris, île-de-france, france
False
['paypal']
usd
None
None
None

[req] ($60 ) - (#brewton, al, usa) (5/3/18), (paypal)
brewton, al, usa
False
['paypal']
usd
60
None
5/3

[req] (1500 ) - (#colorado springs, colorado, united states), (april 20th, 2018), (paypal, moneygram/pre-arranged)
colorado springs, colorado, united states
True
['paypal']
usd


7/21

[req] ($25 ) - (#hamilton, ontario, canada), (may 31), (paypal or e-transfer)
hamilton, ontario, canada
False
['paypal']
usd
None
None
None

[req] (£1500) - (#wolverhampton, west-midlands, uk), (5 months), (paypal- 350 a month)
wolverhampton, west-midlands, uk
False
['paypal']
gbp
350
None
None

[req] 400 (#meridian,ms, usa) (payment meant 4/30 and 5/25) (prearranged w/u/readerbore)
meridian,ms, usa
True
[]
usd
400
None
None

[req] (60$) - (#mahwah, nj, usa), (05/20/2018 + 20), (paypal,cashapp)
mahwah, nj, usa
False
['paypal', 'cashapp']
usd
60
None
05/20

[req] (80) - (#kernville, california, usa), (4/21/18), (paypal)
kernville, california, usa
False
['paypal']
usd
80
None
4/21

[req]($1000)(#sheboygan, wi, usa)(repay $1150 on 5/8/18)(prearranged/paypal)
sheboygan, wi, usa
True
['paypal']
usd
1000
1150
5/8

[req] ($525) - (#minneapolis, mn, usa), (by 5/4/18), (paypal)
minneapolis, mn, usa
False
['paypal']
usd
525
None
5/4

[req] ($600) - (#san antonio, tx, usa), (repayment of $2

...

usd
300
350
None

[req] ($150) - (#portland, or, us), (08/15), (venmo or paypal)
portland, or, us
False
['paypal', 'venmo']
usd
150
None
08/15

[req] ($300)-(#asheboro, nc, usa),($200 08/10/2018 $200 08/17/2018), (paypal)
asheboro, nc, usa
False
['paypal']
usd
None
None
None

[req] ($250) - (#desmoines, iowa, usa), ($350 by aug. 17) (paypal, cash app, circle)
desmoines, iowa, usa
False
['paypal', 'cashapp']
usd
None
None
None

[req] ($75) (#san jose, california, united states) (int. negotiable on 08/31/18) (paypal)
san jose, california, united states
False
['paypal']
usd
75
None
08/31

[req] ( $300) - (#gainesville, fl, usa), (08/04/2018), (paypal/pre-arranged).
gainesville, fl, usa
True
['paypal']
usd
300
None
08/04

[req] (110£ ) - (#barnsley, uk), (120£ on 03/08/2018), (paypal/bacs)
barnsley, uk
False
['paypal']
gbp
110
120
03/08

[req] ($225) - (#salt lake city, utah, usa), ($275 by aug. 17) (paypal, venmo)
salt lake city, utah, usa
False
['paypal', 'venmo']
usd
225
275
None

[req]

['paypal']
usd
None
None
8/10

[req] ($100) - (#milam, tx, usa), ($50 8/15 and $50 8/29), (paypal)
milam, tx, usa
False
['paypal']
usd
100
None
None

[req] ($810) - (#mechanicsburg,pa,usa), (1/18 and 1/25), (paypal)
mechanicsburg,pa,usa
False
['paypal']
usd
810
None
None

[req] ($500) - (#ankeny, iowa, usa), (09/30/2018), (paypal)
ankeny, iowa, usa
False
['paypal']
usd
500
None
09/30

[req] ($600) - (#phoenix, az, usa), (9/22/2018 + $60 interest), (paypal)
phoenix, az, usa
False
['paypal']
usd
600
None
9/22

[req] ($25) (#west chester, pa, usa) (repay $30 tomorrow, 7/31/2018) (zelle or paypal)
west chester, pa, usa
False
['paypal', 'zelle']
usd
None
None
7/31

[req] ($300), (# new haven, connecticut, usa), (8/1), (paypal), (pre-arranged)
 new haven, connecticut, usa
True
['paypal']
usd
300
None
8/1

[req] ($2100)-(# clifton, nj, usa),($2500) 10/28/2018 (paypal)
 clifton, nj, usa
False
['paypal']
usd
None
None
10/28

[req]($40)(#orlando , florida)($50 on 8/8/18)(paypal)
orlando , florid

eur
None
None
None

[req] ($175 repay 200) - (#houston, texas, usa) (8/17), (paypal)
houston, texas, usa
False
['paypal']
usd
175
200
8/17

[req] (260) - (#chicago, il, us), ($310 on 8/27), (paypal)
chicago, il, us
False
['paypal']
usd
260
310
8/27

[req] ($35 ) - (#phoenix, az, usa), ($45 august 7th), (paypal)
phoenix, az, usa
False
['paypal']
usd
None
None
None

[req] ($200) - (#woodstock, georgia, usa), (8/15/18), (paypal)
woodstock, georgia, usa
False
['paypal']
usd
200
None
8/15

[req]($200)(#lexington, ky, usa)(paypal, cashapp)
lexington, ky, usa
False
['paypal', 'cashapp']
usd
200
None
None

[req] ($250) (#anchorage, alaska, usa) (paypal)
anchorage, alaska, usa
False
['paypal']
usd
250
None
None

[req] ($800.00)-(#chesterfield, mi usa), ($1000.00 on or before 10/31/18) (cash app)
chesterfield, mi usa
False
['cashapp']
usd
None
None
10/31

[req] ($150) (#ottawa, ontario, canada) (08/24 $200 payback)
ottawa, ontario, canada
False
[]
usd
None
None
08/24

[req] ($250) - (#columbus, 

[req] ($205) - (#columbia, missouri, usa), ($230 sept 6th), (paypal/transferwise)
columbia, missouri, usa
False
['paypal']
usd
205
230
None

[req] ($400) - (#san diego, ca, usa) ($550 in payments) (pre-arranged)
san diego, ca, usa
True
[]
usd
None
None
None

[req] ($100) - (#fresno, ca, usa), (8/24/2018 will pay back $125), (paypal/cashapp)
fresno, ca, usa
False
['paypal', 'cashapp']
usd
None
None
8/24

[req] ($600) - (#austin, tx, usa), (12/31/18), (paypal)
austin, tx, usa
False
['paypal']
usd
600
None
12/31

[req] ($500) - (#lakewood, co, usa), (will pay back $600 on 8/15/ 2018), (paypal)
lakewood, co, usa
False
['paypal']
usd
500
600
8/15

[req] ($45) - (#jonesboro, ga), ($55 on 8/9), (cash app/paypal)
jonesboro, ga
False
['paypal', 'cashapp']
usd
None
None
8/9

[req] (300$ ) - (#norcross, ga, usa), (8/10/18), (cash app).
norcross, ga, usa
False
['cashapp']
usd
300
None
8/10

[req] (400) - (#atlanta, ga, usa), (450 by 10/14 3 monthly payments), (paypal/simple)
atlanta, ga, usa
False

west chester, pa, usa
False
['paypal']
usd
None
None
9/28

[req] ($45) - (#glendale, arizona, usa) (return $55 on august 16th) (paypal)
glendale, arizona, usa
False
['paypal']
usd
None
None
None

[req] ($400) - (#austin, tx, usa), (7/31/18), (paypal)
austin, tx, usa
False
['paypal']
usd
400
None
7/31

[req] ($200) - (#pittsburgh, pa, usa) (paypal, venmo, cashapp)
pittsburgh, pa, usa
False
['paypal', 'venmo', 'cashapp']
usd
200
None
None

[req] (105) - (#lisbon, pt), (130 - 31/08), (paypal)
lisbon, pt
False
['paypal']
usd
105
130
None

[req]($80) - (#orlando, fl, usa)(08/31 $100)(venmo)
orlando, fl, usa
False
['venmo']
usd
None
None
08/31

[req] ($50) - (#portsmouth, oh, united states), ($75 08/13.2018), (paypal/moneygram)
portsmouth, oh, united states
False
['paypal']
usd
None
None
08/13

[req]($80) - (#mesa, az, usa) (08/17 $100) (venmo, cashapp)
mesa, az, usa
False
['venmo', 'cashapp']
usd
None
None
08/17

[req] (£600 ) - (#newcastle, england, uk), (repay £750 saturday 1st august),